In [1]:
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"../Algorithm encapsulation"))
sys.path.append(os.path.join(os.getcwd(),".."))
print(sys.path)
from MPband import MPband
import pandas as pd
from ga_platoon import max_dt

['c:\\Users\\Administrator\\Desktop\\速度爬取\\Case Analysis', 'd:\\Anaoconda\\envs\\cplex\\python37.zip', 'd:\\Anaoconda\\envs\\cplex\\DLLs', 'd:\\Anaoconda\\envs\\cplex\\lib', 'd:\\Anaoconda\\envs\\cplex', '', 'C:\\Users\\Administrator\\AppData\\Roaming\\Python\\Python37\\site-packages', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\requests-2.22.0-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\docloud-1.0.375-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\docplex-2.11.176-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\chardet-3.0.4-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\idna-2.8-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\urllib3-1.25.11-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\cplex-12.10.0.0-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\win32', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\win32\\lib', 'd:\\Anaocond

In [2]:
def get_rf(d,p):
    tmp=[]
    for i,a in enumerate(p):
        idx=np.where(a!=0)[0]
        a=a&0
        if len(idx)>0:
            a[:idx[0]]=1
        tmp.append((d[i]*a).sum())
    return tmp

In [3]:
phase = np.loadtxt("./data/phase")
cir=phase[0].sum()
phase=phase/cir
vol = np.loadtxt("./data/volume")
qv = np.loadtxt("./data/path_vol")
phase_get = np.load("./data/phase_get.npy")
d = np.loadtxt("./data/distance")
straight_get = np.load("./data/straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])
px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
pxb = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
qb=[120,120]
lower, upper = 4, 10
linspace = np.arange(lower, upper, 1)
lin_num = len(linspace)

# 南北直行方向绿灯时间
sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
nump = len(phase_get)
M = 1e6
nx = 0.00000001
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])
r=1-g
rf=np.array([get_rf(phase,phase_get[i]) for i in range(numr)])

ex=[1,0,1,0,1,0,1,1,1,1,1,0]
dwt=15
qb_x=np.ones_like(sg)*20
cap=np.ones_like(sg,dtype=int)*3

spd_on=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","上行速度数据.csv"),index_col=0)
spd_in=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","下行速度数据.csv"),index_col=0)

p=[0,0,0,0,1,0,0,0,1,0,1,0,0]

In [4]:
mpb=MPband(phase, [80,180], vol, qv, phase_get, d, straight_get, ison, px, pxb, qb, lower, upper, 
            be, speedVar[0], speedVar[1],ex, dwt,lower,upper,speedVar[0], speedVar[1],p)
mpb._solve()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 2 times.
MIP Presolve eliminated 669 rows and 209 columns.
MIP Presolve modified 689 coefficients.
Aggregator did 50 substitutions.
Reduced MIP has 605 rows, 333 columns, and 1920 nonzeros.
Reduced MIP has 19 binaries, 69 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.13 ticks)
Probing fixed 0 vars, tightened 51 bounds.
Probing time = 0.00 sec. (0.09 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 187 coefficients.
Reduced MIP has 605 rows, 333 columns, and 1920 nonzeros.
Reduced MIP has 19 binaries, 69 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.05 ticks)
Probing time = 0.00 sec. (0.08 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root rel

In [5]:
a=mpb.get_dataframe()
a

,cross_number,distance,b1,b2,b3,b4,b5,b6,offset,p,...,u3,u4,u5,u6,bb1,bb2,dw1,dw2,tb1,tb2
0,1.0,800.0,33.076692,33.076692,0.000000,38.305126,0.000000,38.706767,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,33.076692,38.706767,15.0,15.0,117.293233,98.174436
1,2.0,520.0,21.933835,21.933835,0.000000,38.706767,0.000000,38.706767,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,21.933835,43.163910,15.0,15.0,52.000000,97.607519
2,3.0,500.0,21.933835,21.933835,0.000000,23.458647,0.000000,23.458647,57.571429,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,21.933835,23.458647,15.0,15.0,65.000000,74.989474
3,4.0,490.0,32.490226,32.490226,0.000000,23.458647,0.000000,23.458647,117.293233,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,32.490226,32.490226,15.0,15.0,116.003006,117.293233
4,5.0,370.0,8.000000,0.000000,63.838710,35.919355,0.000000,35.919355,140.860215,1.0,...,1.822120e-01,5.108974e-03,1.000000e-08,5.108974e-03,35.919355,35.919355,15.0,15.0,84.981183,83.698925
5,6.0,254.0,34.229032,0.000000,34.229032,35.919355,0.000000,35.919355,29.341398,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,34.229032,91.559140,15.0,15.0,49.540323,63.500000
6,7.0,585.0,23.241935,0.000000,23.241935,25.834397,0.000000,34.229032,135.366667,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,23.241935,34.229032,15.0,15.0,140.860215,140.860215
7,8.0,1020.0,23.241935,0.000000,23.241935,12.254839,0.000000,34.229032,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,23.241935,23.241935,15.0,15.0,140.860214,140.860215
8,9.0,409.0,49.625514,0.000000,49.625514,49.625514,49.625514,0.000000,155.454421,1.0,...,1.000000e-08,1.000000e-08,4.349997e-02,1.000000e-08,49.625514,49.625514,15.0,15.0,57.534164,108.994407
9,10.0,547.0,63.114329,0.000000,63.114329,49.625514,49.625514,0.000000,39.715607,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,63.114329,63.114329,15.0,15.0,106.070904,69.700000


In [7]:
a[["b"+str(i+1) for i in range(6)]].mean()

b1    37.469795
b2     8.418045
b3    33.347035
b4    41.809858
b5    23.820848
b6    20.355969
dtype: float64